In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [3]:
input_folder = '../data/railways/sound_transit/'
input_file = input_folder+'LINKLine.shp'

output_folder = '../data/output_images/SeattleLLR/'
output_csv = output_folder+'SEA.csv'

update_folder = output_folder+'/set_3/'

### Import Routes

In [4]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4269'})

routes.head()

,OBJECTID,LINK_TYPE,DESCRIPTIO,STATUS,Shape_Leng,geometry
0,68,1,Central Link,COMPLETE,73476.308611,(LINESTRING (-122.2886826905774 47.46424879456...
1,101,4,Airport Link,COMPLETE,9028.515287,LINESTRING (-122.296751670065 47.4444939187097...
2,109,2,Tacoma Link,COMPLETE,9381.691809,LINESTRING (-122.4400288855092 47.256333855845...
3,113,6,University Link,CONST,16875.317490,LINESTRING (-122.3311232117972 47.613775582999...
4,130,3,Northgate Link,CONST,22284.085571,LINESTRING (-122.3281386191352 47.705615661930...


In [5]:
routes.shape

(6, 6)

In [10]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
            for j in range(0,len(g)):
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

3973 points


In [6]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

1018 points


In [11]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,-122.2697075007809_47.47314172665546,-122.269707,47.473142,1
1,-122.27107741516011_47.465236046868256,-122.271077,47.465236,1
2,-122.27291605481625_47.48176886740649,-122.272916,47.481769,1
3,-122.27293724183188_47.479459541696556,-122.272937,47.479460,1
4,-122.27334338167215_47.48344844273076,-122.273343,47.483448,1


In [27]:
'''
Get subselection of points excuding current points
'''
curr_points = list(zip(curr_df['Longitude'].tolist(),curr_df['Latitude'].tolist()))
# print(curr_points)
# print('\n')

route_points = []
for point in range(0,len(points),40):
    tmp = points[point]
    if tmp not in curr_points:
#         print(tmp)
        route_points.append(tmp)

print(len(route_points))

100


In [15]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,-122.2886826905774_47.464248794560376,-122.288683,47.464249
1,-122.28559327844411_47.46322876704172,-122.285593,47.463229
2,-122.28460363170346_47.463241793803235,-122.284604,47.463242
3,-122.28342496740648_47.46344488032227,-122.283425,47.463445
4,-122.28234511970552_47.46382419989612,-122.282345,47.463824


In [30]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)
# df.to_csv('tmp.csv',header=True,index=None)

In [29]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 12
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [23]:
'''
Get satellite preview for image
'''

row = 1

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [17]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 

In [ ]:
root = os.path.dirname()